In [4]:
! cd ~
! hdfs dfs -mkdir -p  hdfs://localhost:9000/user/ashok/data_files/EasyVisa
! hdfs dfs -put /home/ashok/Downloads/EasyVisa.csv hdfs://localhost:9000/user/ashok/data_files/EasyVisa
! hdfs dfs -put /home/ashok/Downloads/EasyVisa.csv hdfs://localhost:9000/user/ashok/data_files/EasyVisa
! hdfs dfs -ls -h hdfs://localhost:9000/user/ashok/data_files/EasyVisa


put: `hdfs://localhost:9000/user/ashok/data_files/EasyVisa/EasyVisa.csv': File exists
Found 1 items
-rw-r--r--   1 ashok supergroup      1.8 M 2023-10-14 06:30 hdfs://localhost:9000/user/ashok/data_files/EasyVisa/EasyVisa.csv


In [5]:
import numpy as np

# 1.1   For transforming categorical data to integer and to dummy
from pyspark.ml.feature import  StringIndexer, OneHotEncoder

# 1.2   For collecting all features at one place
#       A feature transformer that merges multiple columns into one vector column.
from pyspark.ml.feature import VectorAssembler

# 1.3 Minmax scaler
from pyspark.ml.feature import MinMaxScaler, StandardScaler

# 1.4   To execute all transformation operations as pipeline
from pyspark.ml import Pipeline

# 1.5 Logistic Regression modeling
from pyspark.ml.classification import LogisticRegression

# 1.6 Parameter grid builder and cross-validator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# 1.7 Evaluate classification. Default AUC.
#     Change parameter 'metricName' for others
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# 1.8 
from pyspark.sql.functions import col

# 1.9 Misc
import time

In [6]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [7]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_4846/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [9]:
df = spark.read.csv(
                      path = "/user/ashok/data_files/EasyVisa",
                      sep = ",",
                      inferSchema = True,
                      ignoreLeadingWhiteSpace = True,
                      ignoreTrailingWhiteSpace= True,
                      header = True,
                      nullValue = "?"           
                      
                      )

df.show(2)


+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
|case_id|continent|education_of_employee|has_job_experience|requires_job_training|no_of_employees|yr_of_estab|region_of_employment|prevailing_wage|unit_of_wage|full_time_position|case_status|
+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
| EZYV01|     Asia|          High School|                 N|                    N|          14513|       2007|                West|       592.2029|        Hour|                 Y|     Denied|
| EZYV02|     Asia|             Master's|                 Y|                    N|           2412|       2002|           Northeast|       83425.65|        Year|                 Y|  Certified|
+-------+---------+---------------------

In [10]:
# Save df's initial column list
cols_original = df.columns
cols_original


# DataFrame dtypes
df.dtypes

['case_id',
 'continent',
 'education_of_employee',
 'has_job_experience',
 'requires_job_training',
 'no_of_employees',
 'yr_of_estab',
 'region_of_employment',
 'prevailing_wage',
 'unit_of_wage',
 'full_time_position',
 'case_status']

[('case_id', 'string'),
 ('continent', 'string'),
 ('education_of_employee', 'string'),
 ('has_job_experience', 'string'),
 ('requires_job_training', 'string'),
 ('no_of_employees', 'int'),
 ('yr_of_estab', 'int'),
 ('region_of_employment', 'string'),
 ('prevailing_wage', 'double'),
 ('unit_of_wage', 'string'),
 ('full_time_position', 'string'),
 ('case_status', 'string')]

In [15]:
df.select('case_status').distinct().show()

+-----------+
|case_status|
+-----------+
|     Denied|
|  Certified|
+-----------+



In [16]:
df.count()

25480

In [17]:
dx = df.groupby('case_status').count()
dx.show()

+-----------+-----+
|case_status|count|
+-----------+-----+
|     Denied| 8462|
|  Certified|17018|
+-----------+-----+



In [18]:
# Per column how many null values:

from pyspark.sql.functions import isnan, when, count, col

# Create function to discover null values:

def null_values(data):
  data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [19]:
# Understanding the above function--step-by-step
# a
df.select([isnan(c)  for c in df.columns]).show(2)
# b
df.select([ (isnan(c)|  col(c).isNull()) for c in df.columns]).show(2)
# c
df.select([ (isnan(c)|  col(c).isNull()).alias(c)  for c in df.columns]).show(2)
# d
df.select([ when ( ( isnan(c) |  col(c).isNull()), c)  for c in df.columns]).show(2)
# e
df.select([ count(when ( ( isnan(c) |  col(c).isNull()), c))  for c in df.columns]).show(2)


+--------------+----------------+----------------------------+-------------------------+----------------------------+----------------------+------------------+---------------------------+----------------------+-------------------+-------------------------+------------------+
|isnan(case_id)|isnan(continent)|isnan(education_of_employee)|isnan(has_job_experience)|isnan(requires_job_training)|isnan(no_of_employees)|isnan(yr_of_estab)|isnan(region_of_employment)|isnan(prevailing_wage)|isnan(unit_of_wage)|isnan(full_time_position)|isnan(case_status)|
+--------------+----------------+----------------------------+-------------------------+----------------------------+----------------------+------------------+---------------------------+----------------------+-------------------+-------------------------+------------------+
|         false|           false|                       false|                    false|                       false|                 false|             false|             

In [20]:
null_values(df)

+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
|case_id|continent|education_of_employee|has_job_experience|requires_job_training|no_of_employees|yr_of_estab|region_of_employment|prevailing_wage|unit_of_wage|full_time_position|case_status|
+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
|      0|        0|                    0|                 0|                    0|              0|          0|                   0|              0|           0|                 0|          0|
+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+



In [22]:
# Use where filter and isNull()
df.select('*').where(df.case_status.isNull()).count()

0

In [24]:
df.select('*').where(~df.case_status.isNull()).count()

25480

In [25]:
# MODELLING PIPELINE
x = df.dtypes
x

[('case_id', 'string'),
 ('continent', 'string'),
 ('education_of_employee', 'string'),
 ('has_job_experience', 'string'),
 ('requires_job_training', 'string'),
 ('no_of_employees', 'int'),
 ('yr_of_estab', 'int'),
 ('region_of_employment', 'string'),
 ('prevailing_wage', 'double'),
 ('unit_of_wage', 'string'),
 ('full_time_position', 'string'),
 ('case_status', 'string')]

In [26]:
# What are my column names:

cat_cols= [ c[0]  for c in x if c[1] == "string" ]
cat_cols      

len(cat_cols)    

['case_id',
 'continent',
 'education_of_employee',
 'has_job_experience',
 'requires_job_training',
 'region_of_employment',
 'unit_of_wage',
 'full_time_position',
 'case_status']

9

In [27]:
cat_cols= cat_cols[0:9]

In [28]:
num_cols = [ i[0]   for i in df.dtypes  if i[1] != "string" ]
num_cols

['no_of_employees', 'yr_of_estab', 'prevailing_wage']

In [29]:
# After String indexing, names of our
#        cat_columns would be:

cat_cols_si = [c+"_index"  for c in cat_cols]
cat_cols_si

['case_id_index',
 'continent_index',
 'education_of_employee_index',
 'has_job_experience_index',
 'requires_job_training_index',
 'region_of_employment_index',
 'unit_of_wage_index',
 'full_time_position_index',
 'case_status_index']

In [30]:
ohe_cols = [ c + "_ohe"   for  c  in cat_cols ]
ohe_cols

['case_id_ohe',
 'continent_ohe',
 'education_of_employee_ohe',
 'has_job_experience_ohe',
 'requires_job_training_ohe',
 'region_of_employment_ohe',
 'unit_of_wage_ohe',
 'full_time_position_ohe',
 'case_status_ohe']

In [31]:
# Getting mode of a feature--step-by-step

df.groupby('continent').count().show(3)
df.groupby('continent').count().orderBy("count").show(3)
df.groupby('continent').count().orderBy("count",ascending = False).show(3)
df.groupby('continent').count().orderBy("count",ascending = False).first()   

# Row object:
df.groupby('continent').count().orderBy("count",ascending = False).first()['continent'] 
df.groupby('continent').count().orderBy("count",ascending = False).first().continent    
df.groupby('continent').count().orderBy("count",ascending = False).first()[0]           

+-------------+-----+
|    continent|count|
+-------------+-----+
|       Europe| 3732|
|       Africa|  551|
|North America| 3292|
+-------------+-----+
only showing top 3 rows

+-------------+-----+
|    continent|count|
+-------------+-----+
|      Oceania|  192|
|       Africa|  551|
|South America|  852|
+-------------+-----+
only showing top 3 rows

+-------------+-----+
|    continent|count|
+-------------+-----+
|         Asia|16861|
|       Europe| 3732|
|North America| 3292|
+-------------+-----+
only showing top 3 rows



Row(continent='Asia', count=16861)

'Asia'

'Asia'

'Asia'

In [32]:
[[i,df.groupby(i).count().orderBy("count", ascending=False).first()[0]] for i in df.columns]

[['case_id', 'EZYV888'],
 ['continent', 'Asia'],
 ['education_of_employee', "Bachelor's"],
 ['has_job_experience', 'Y'],
 ['requires_job_training', 'N'],
 ['no_of_employees', 183],
 ['yr_of_estab', 1998],
 ['region_of_employment', 'Northeast'],
 ['prevailing_wage', 138.76],
 ['unit_of_wage', 'Year'],
 ['full_time_position', 'Y'],
 ['case_status', 'Certified']]

In [33]:
null_values(df)

+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
|case_id|continent|education_of_employee|has_job_experience|requires_job_training|no_of_employees|yr_of_estab|region_of_employment|prevailing_wage|unit_of_wage|full_time_position|case_status|
+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+
|      0|        0|                    0|                 0|                    0|              0|          0|                   0|              0|           0|                 0|          0|
+-------+---------+---------------------+------------------+---------------------+---------------+-----------+--------------------+---------------+------------+------------------+-----------+



In [60]:

label_To_integer = StringIndexer(
                                   inputCol = "case_status",
                                   outputCol = "label"
                                 )

# 6.4.1
model = label_To_integer.fit(df.select("case_status"))
model.transform(df.select('case_status')).take(10)

[Row(case_status='Denied', label=1.0),
 Row(case_status='Certified', label=0.0),
 Row(case_status='Denied', label=1.0),
 Row(case_status='Denied', label=1.0),
 Row(case_status='Certified', label=0.0),
 Row(case_status='Certified', label=0.0),
 Row(case_status='Certified', label=0.0),
 Row(case_status='Denied', label=1.0),
 Row(case_status='Certified', label=0.0),
 Row(case_status='Certified', label=0.0)]

In [61]:
#PIPELINE

In [62]:


pipe = Pipeline(
                stages =  [
                            # StringIndex target
                            StringIndexer(
                                           inputCol ="case_status",
                                           outputCol = "label"
                                           ),

                             # StringIndex all categorical cols
                             StringIndexer(
                                          inputCols = cat_cols,
                                          outputCols = cat_cols_si
                                         ),
                    
                             # OHE, StringIndexed columns
                             OneHotEncoder(
                                           inputCols = cat_cols_si,
                                           outputCols = ohe_cols
                                          ),
                    
                              # Assemble OHE + num_cols
                              VectorAssembler
                                           (
                                           inputCols = ohe_cols + num_cols,
                                           outputCol = 'ass_features'    

                                           ),
                             
                             #  Scale 'vector' column
                             StandardScaler(
                                            inputCol = 'ass_features',
                                            outputCol = 'features'

                                           )
                     
                       ]

            )

In [63]:

model= pipe.fit(df)

df_trans = model.transform(df)

2023-10-14 07:30:28,899 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.6 MiB
2023-10-14 07:30:29,311 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.6 MiB


In [64]:

print(df_trans.columns)


len(df.columns)         
len(df_trans.columns)   

['case_id', 'continent', 'education_of_employee', 'has_job_experience', 'requires_job_training', 'no_of_employees', 'yr_of_estab', 'region_of_employment', 'prevailing_wage', 'unit_of_wage', 'full_time_position', 'case_status', 'label', 'case_id_index', 'continent_index', 'education_of_employee_index', 'has_job_experience_index', 'requires_job_training_index', 'region_of_employment_index', 'unit_of_wage_index', 'full_time_position_index', 'case_status_index', 'case_id_ohe', 'continent_ohe', 'education_of_employee_ohe', 'has_job_experience_ohe', 'requires_job_training_ohe', 'region_of_employment_ohe', 'unit_of_wage_ohe', 'full_time_position_ohe', 'case_status_ohe', 'ass_features', 'features']


12

33

In [65]:
# 8.0 In building Pipeline,
#      keyword 'stages' is a must

pipe = Pipeline(
                stages =  [
                            # 11.0.1 StringIndex target
                            StringIndexer(
                                           inputCol ="income",
                                           handleInvalid = 'keep',
                                           outputCol = "label"
                                           ),

                             # 11.0.2 StringIndex all categorical cols
                             StringIndexer(
                                          inputCols = cat_cols,
                                          handleInvalid = 'keep',
                                          outputCols = cat_cols_si
                                         ),
                    
                             # 11.0.3 OHE, StringIndexed columns
                             OneHotEncoder(
                                           inputCols = cat_cols_si,
                                           outputCols = ohe_cols
                                          ),
                    
                              # 11.0.4 Assemble OHE + num_cols
                              VectorAssembler
                                           (
                                           inputCols = ohe_cols + num_cols,
                                           outputCol = 'ass_features'    

                                           ),
                             
                             # 11.0.5 Scale 'vector' column
                             StandardScaler(
                                            inputCol = 'ass_features',
                                            outputCol = 'features'
                                           ),
                            
                             # 11.0.6 Modeling
                             # https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegression
                             LogisticRegression(
                                 
                                                 featuresCol='features',
                                                 labelCol='label',
                                                 maxIter=30
                                               )
                    
                       ]

            )

In [66]:
# train/test split
train, test = df.randomSplit([0.8, 0.2], seed=12345)

In [67]:
train.count() 

20272

In [68]:
start = time.time()
lrModel = pipe.fit(train)
end = time.time()
(end-start)/60

Py4JJavaError: An error occurred while calling o1845.fit.
: org.apache.spark.SparkException: Input column income does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spark.m

In [69]:
#PREDICTIONS


predictions = lrModel.transform(test)


print()
print(predictions.columns)      


print()
predictions.printSchema()



['case_id', 'continent', 'education_of_employee', 'has_job_experience', 'requires_job_training', 'no_of_employees', 'yr_of_estab', 'region_of_employment', 'prevailing_wage', 'unit_of_wage', 'full_time_position', 'case_status', 'label', 'case_id_index', 'continent_index', 'education_of_employee_index', 'has_job_experience_index', 'requires_job_training_index', 'region_of_employment_index', 'unit_of_wage_index', 'full_time_position_index', 'case_status_index', 'case_id_ohe', 'continent_ohe', 'education_of_employee_ohe', 'has_job_experience_ohe', 'requires_job_training_ohe', 'region_of_employment_ohe', 'unit_of_wage_ohe', 'full_time_position_ohe', 'case_status_ohe', 'ass_features', 'features']

root
 |-- case_id: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- education_of_employee: string (nullable = true)
 |-- has_job_experience: string (nullable = true)
 |-- requires_job_training: string (nullable = true)
 |-- no_of_employees: integer (nullable = true)
 |-- yr_o

In [70]:
selected = predictions.select("label", "continent", "unit_of_wage", "prevailing_wage", "case_status")
selected.show(3)

+-----+---------+------------+---------------+-----------+
|label|continent|unit_of_wage|prevailing_wage|case_status|
+-----+---------+------------+---------------+-----------+
|  0.0|     Asia|        Year|       53635.39|  Certified|
|  0.0|     Asia|        Hour|       849.9988|  Certified|
|  0.0|     Asia|        Year|       14130.76|  Certified|
+-----+---------+------------+---------------+-----------+
only showing top 3 rows



In [85]:
from pyspark.mllib.evaluation import MulticlassMetrics
preds_and_labels = predictions.select(['case_status','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/opt/spark-3.2.3-bin-hadoop2.7/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
2023-10-14 07:40:16,269 ERROR executor.Executor: Exception in task 0.0 in stage 139.0 (TID 98)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/pyspark/sql/session.py", line 682, in prepare
    verify_f

Py4JJavaError: An error occurred while calling o8405.confusionMatrix.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 139.0 failed 1 times, most recent failure: Lost task 0.0 in stage 139.0 (TID 98) (master executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/opt/spark-3.2.3-bin-hadoop2.7/python/pyspark/sql/session.py", line 682, in prepare
    verify_f

In [81]:
# Create ParamGrid for Cross Validation
#     Ref: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder
#     Ref: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator

#  K-fold cross validation performs model selection by splitting the dataset into a set of
#    non-overlapping randomly partitioned folds which are used as separate training and
#     test datasets e.g., with k=3 folds, K-fold cross validation will generate 3 (training, test)
#      dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing.
#       Each fold is used as the test set exactly once.

# One Way

lr = pipe.getStages()[5]



grid = (
        ParamGridBuilder()
                 .addGrid(lr.regParam, [ 0.01, 1, 2.0]) \
                 .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
                 .addGrid(lr.maxIter, [1, 10]) \
                 .build()
       )


In [82]:
# Define our evaluator
evaluator = BinaryClassificationEvaluator()

# And crossvalidator
cv = CrossValidator(estimator= pipe, \
                    estimatorParamMaps= grid,     \
                    evaluator=evaluator,  \
                    numFolds=3
                    )

In [84]:

start = time.time()
cvModel = cv.fit(train)
end = time.time()
print (((end-start)/60) , "minutes")

TypeError: 'CrossValidator' object is not callable